In [1]:
import os

os.chdir("/scratch/project/tcr_neoantigen/scripts")

from pathlib import Path
from pyfaidx import Fasta

from _utils import (
    read_and_filter,
    create_result_list,
    extract_result,
    extract_exon_info,
    filter_exon_pos,
    generate_windows,
    get_sequences_insertion,
    complementary_sequence,
    reverse_complement,
    flanking_lower_positions,
    # print_windows,
)

# define head paths
HG38FOLDER = Path("/scratch/project/tcr_neoantigen/resources/references/hg38")
INPUTFOLDER = Path(
    "/scratch/project/tcr_neoantigen/results/cSCC_BC_seq_data_10_patients/nextNEOpi"
)

In [2]:
# import reference and data
fasta_file = HG38FOLDER / "gdc" / "GRCh38.d1.vd1" / "fasta" / "GRCh38.d1.vd1.fa"
gtf_file = HG38FOLDER / "annotation" / "gencode.v33.primary_assembly.annotation.gtf"
refgen = Fasta(filename=fasta_file)
exon_info = extract_exon_info(gtf_file)

In [3]:
samples = [
    # "2020135",
    # "2020239_WO1",
    # "2020246_NO1",
    # "2020260_WO1",
    # "2020281_WO1",
    # "2021111_MO1",
    "DES001",
    # "DES002",
    # "DES002_001",
    # "DES002_002",
    # "DES010",
]

In [4]:
results = {}
for sample in samples:
    file_input_path = (
        INPUTFOLDER
        / sample
        / "analyses"
        / sample
        / "05_vep"
        / "tables"
        / "high_confidence"
        / f"{sample}_hc_vep.txt"
    )
    results[sample] = read_and_filter(file_input_path)

for sample in samples:
    print(sample, results[sample].shape)

for sample in samples:
    print(sample, results[sample].VARIANT_CLASS.unique())

DES001 (4164, 91)
DES001 ['SNV' 'substitution' 'deletion' 'insertion']


In [5]:
df = results["DES001"].copy()

In [6]:
mut_dict = create_result_list(df)

In [7]:
from _utils import find

find(mut_dict["variant_class"], "insertion")

[2670, 2694, 2722, 3923]

In [40]:
find(mut_dict["variant_class"], "deletion")

[59,
 464,
 662,
 927,
 1061,
 1243,
 1959,
 2020,
 2323,
 2332,
 2333,
 2334,
 2568,
 2687,
 3110,
 3197,
 3262]

In [9]:
# find(mut_dict["variant_class"], "substitution")

In [15]:
for mut in [2670, 2694, 2722, 3923]:
    print(extract_result(mut_dict, mut))

{'reference': '-', 'variant': 'GCC', 'amino_acid_ref': 'L', 'amino_acid_var': 'LP', 'codon_ref': 'ctg', 'codon_var': 'ctGCCg', 'chromosome': 'chr12', 'mutation_location': 131828568, 'mutation_location2': 131828569, 'variant_class': 'insertion', 'gene_id': 'ENSG00000198598', 'protein_id': 'ENSP00000353767', 'transcript_id': 'ENST00000360564', 'gene_symbol': 'MMP17', 'strand': '1'}
{'reference': '-', 'variant': 'G', 'amino_acid_ref': 'S', 'amino_acid_var': 'SX', 'codon_ref': 'tca', 'codon_var': 'tcGa', 'chromosome': 'chr13', 'mutation_location': 32379450, 'mutation_location2': 32379451, 'variant_class': 'insertion', 'gene_id': 'ENSG00000139618', 'protein_id': 'ENSP00000369497', 'transcript_id': 'ENST00000380152', 'gene_symbol': 'BRCA2', 'strand': '1'}
{'reference': '-', 'variant': 'GC', 'amino_acid_ref': '-', 'amino_acid_var': 'X', 'codon_ref': '-', 'codon_var': 'GC', 'chromosome': 'chr13', 'mutation_location': 52844775, 'mutation_location2': 52844776, 'variant_class': 'insertion', 'gene

In [41]:
extract_result(mut_dict, 59)

{'reference': 'AAA',
 'variant': '-',
 'amino_acid_ref': 'IY',
 'amino_acid_var': 'N',
 'codon_ref': 'aTTTat',
 'codon_var': 'aat',
 'chromosome': 'chr1',
 'mutation_location': 19155665,
 'mutation_location2': 19155667,
 'variant_class': 'deletion',
 'gene_id': 'ENSG00000127481',
 'protein_id': 'ENSP00000364403',
 'transcript_id': 'ENST00000375254',
 'gene_symbol': 'UBR4',
 'strand': '-1'}

In [17]:
mut_info = extract_result(mut_dict, 2694)
mut_info

{'reference': '-',
 'variant': 'G',
 'amino_acid_ref': 'S',
 'amino_acid_var': 'SX',
 'codon_ref': 'tca',
 'codon_var': 'tcGa',
 'chromosome': 'chr13',
 'mutation_location': 32379450,
 'mutation_location2': 32379451,
 'variant_class': 'insertion',
 'gene_id': 'ENSG00000139618',
 'protein_id': 'ENSP00000369497',
 'transcript_id': 'ENST00000380152',
 'gene_symbol': 'BRCA2',
 'strand': '1'}

In [18]:
ref_seqs, var_seqs = get_sequences_insertion(
    mut_info=mut_info,
    exon_info=exon_info,
    fasta=refgen,
)

AssertionError: 

In [31]:
(
    # ref,
    var,
    # aa_ref,
    # aa_var,
    codon_ref,
    # codon_var,
    chrom,
    mut,
    mut2,
    # var_class,
    gene,
    # protein,
    # transcript,
    # gene_name,
    strand,
) = (
    # mut_info["reference"],
    mut_info["variant"],
    # mut_info["amino_acid_ref"],
    # mut_info["amino_acid_var"],
    mut_info["codon_ref"],
    # mut_info["codon_var"],
    mut_info["chromosome"],
    mut_info["mutation_location"],
    mut_info["mutation_location2"],
    # mut_info["variant_class"],
    mut_info["gene_id"],
    # mut_info["protein_id"],
    # mut_info["transcript_id"],
    # mut_info["gene_symbol"],
    mut_info["strand"],
)
filtered_exon_pos = filter_exon_pos(exon_info[gene], mut, mut_info)
ref_seq_dict, var_seq_dict = {}, {}

if codon_ref == "-":
    # for prot_id in filtered_exon_pos:
    #     ref_seq_dict[prot_id], var_seq_dict[prot_id] = {}, {}
    #     for i, mutx in enumerate([mut, mut2]):
    #         ref_seq_dict[prot_id][i], var_seq_dict[prot_id][i] = defaultdict(
    #             list
    #         ), defaultdict(list)
    #         exon_start_diff = mutx - filtered_exon_pos[prot_id][0][0]
    #         if exon_start_diff >= 47:
    #             left_frames = [45, 46, 47]
    #             frame_dict = {
    #                 45: 45,
    #                 46: 44,
    #                 47: 43,
    #             }
    #         else:
    #             left_frames = [
    #                 exon_start_diff,
    #                 exon_start_diff + 1,
    #                 exon_start_diff + 2,
    #             ]
    #             frame_dict = {
    #                 exon_start_diff: 90 - exon_start_diff,
    #                 exon_start_diff + 1: 90 - exon_start_diff - 1,
    #                 exon_start_diff + 2: 90 - exon_start_diff - 2,
    #             }
#             for j, left_frame in enumerate(left_frames):
#                 # left side
#                 start_seq_loc = mutx - left_frame - 1
#                 left_flank = fasta.get_seq(
#                     chrom,
#                     start_seq_loc,
#                     mutx,
#                 )
#                 # middle
#                 middle_ref = ""
#                 middle_var = var
#                 # right side for reference
#                 end_seq_loc_ref = (
#                     mutx + frame_dict[left_frame]
#                 )  # because the location of the mutation counts as 1
#                 exon_end_diff_ref = filtered_exon_pos[prot_id][0][1] - end_seq_loc_ref
#                 if exon_end_diff_ref < 0:
#                     right_flank_ref = ""
#                     exon_end_diff_ref = (
#                         filtered_exon_pos[prot_id][0][1] - mutx + 1
#                     )  # + 1 for the mutation location itself
#                     end_seq_loc_ref_right_exon = (
#                         filtered_exon_pos[prot_id][1][0]
#                         + frame_dict[left_frame]
#                         - exon_end_diff_ref
#                     )
#                     flank_half1 = fasta.get_seq(
#                         chrom, mutx, filtered_exon_pos[prot_id][0][1]
#                     )
#                     flank_half2 = fasta.get_seq(
#                         chrom,
#                         filtered_exon_pos[prot_id][1][0],
#                         end_seq_loc_ref_right_exon,
#                     )
#                     for half in [flank_half1, flank_half2]:
#                         right_flank_ref += str(half)
#                 else:
#                     right_flank_ref = fasta.get_seq(chrom, mutx, end_seq_loc_ref)
#                 ref_seq1 = str(left_flank) + middle_ref + str(right_flank_ref)
#                 assert len(ref_seq1) == 93
#                 if strand == "-1":
#                     ref_seq_dict[prot_id][i][j].append(complementary_sequence(ref_seq1))
#                 else:
#                     ref_seq_dict[prot_id][i][j].append(ref_seq1)
#                 # right side for variant
#                 end_seq_loc_var = mutx + (
#                     frame_dict[left_frame] - len(var)
#                 )  # this is the pad
#                 exon_end_diff_var = filtered_exon_pos[prot_id][0][1] - end_seq_loc_var
#                 if exon_end_diff_var < 0:
#                     right_flank_var = ""
#                     exon_end_diff_var = (
#                         filtered_exon_pos[prot_id][0][1] - mutx + len(var)
#                     )  # + 1 for the mutation start location
#                     end_seq_loc_var_right_exon = (
#                         filtered_exon_pos[prot_id][1][0]
#                         + frame_dict[left_frame]
#                         - exon_end_diff_var
#                     )
#                     flank_half1 = fasta.get_seq(
#                         chrom, mutx, filtered_exon_pos[prot_id][0][1]
#                     )
#                     flank_half2 = fasta.get_seq(
#                         chrom,
#                         filtered_exon_pos[prot_id][1][0],
#                         end_seq_loc_var_right_exon - len(var),
#                     )
#                     for half in [flank_half1, flank_half2]:
#                         right_flank_var += str(half)
#                 else:
#                     right_flank_var = fasta.get_seq(chrom, mutx, end_seq_loc_var)
#                 var_seq1 = str(left_flank) + middle_var + str(right_flank_var)
#                 assert len(var_seq1) == 93
#                 if strand == "-1":
#                     var_seq_dict[prot_id][i][j].append(complementary_sequence(var_seq1))
#                 else:
#                     var_seq_dict[prot_id][i][j].append(var_seq1)
#                 # get right hang
#                 overlap_ref = get_right_hang(
#                     first_seq=ref_seq1,
#                     chrom=chrom,
#                     prot_id=prot_id,
#                     first_seq_end_loc=end_seq_loc_ref,
#                     filtered_exon_pos=filtered_exon_pos,
#                     fasta=fasta,
#                 )
#                 overlap_var = get_right_hang(
#                     first_seq=var_seq1,
#                     chrom=chrom,
#                     prot_id=prot_id,
#                     first_seq_end_loc=end_seq_loc_var,
#                     filtered_exon_pos=filtered_exon_pos,
#                     fasta=fasta,
#                 )
#                 if strand == "-1":
#                     overlap_ref = [complementary_sequence(s) for s in overlap_ref]
#                     overlap_var = [complementary_sequence(s) for s in overlap_var]
#                 ref_seq_dict[prot_id][i][j] += overlap_ref
#                 var_seq_dict[prot_id][i][j] += overlap_var
# else:
#     # this is now corect but should we add in a codon check?
#     for prot_id in filtered_exon_pos:
#         ref_seq_dict[prot_id], var_seq_dict[prot_id] = defaultdict(list), defaultdict(
#             list
#         )
#         exon_start_diff = mut - filtered_exon_pos[prot_id][0][0]
#         if exon_start_diff >= 47:
#             left_frame = 45
#             left_shifted = False
#         else:
#             left_frame = exon_start_diff
#             left_shifted = True
#         # left side
#         start_seq_loc = mut - left_frame + 1
#         left_flank = fasta.get_seq(
#             chrom,
#             start_seq_loc,
#             mut,
#         )
#         # check the last 3 sequence to ensure that it's the same as codon_ref
#         check_left = (
#             reverse_complement(str(left_flank[-3:]))
#             if strand == "-1"
#             else str(left_flank[-3:])
#         )
#         while check_left != codon_ref.upper():
#             left_frame -= 1
#             start_seq_loc = mut - left_frame + 1
#             left_flank = fasta.get_seq(
#                 chrom,
#                 start_seq_loc,
#                 mut,
#             )
#             if left_shifted:
#                 if left_frame < 42:
#                     break
#             else:
#                 if left_frame < exon_start_diff - 3:
#                     break
#         frame_shift_pad = 45 - left_frame
#         # middle
#         middle_ref = ""
#         # right side for reference
#         end_seq_loc_ref = mut + RIGHT_FRAME + frame_shift_pad
#         exon_end_diff_ref = filtered_exon_pos[prot_id][0][1] - end_seq_loc_ref
#         if exon_end_diff_ref < 0:
#             right_flank_ref = ""
#             exon_end_diff_ref = (
#                 filtered_exon_pos[prot_id][0][1] - mut + 1
#             )  # + 1 for the mutation location itself
#             end_seq_loc_ref_right_exon = (
#                 filtered_exon_pos[prot_id][1][0]
#                 + RIGHT_FRAME
#                 + frame_shift_pad
#                 - exon_end_diff_ref
#                 + 1
#             )
#             flank_half1 = fasta.get_seq(chrom, mut, filtered_exon_pos[prot_id][0][1])
#             flank_half2 = fasta.get_seq(
#                 chrom,
#                 filtered_exon_pos[prot_id][1][0],
#                 end_seq_loc_ref_right_exon,
#             )
#             for half in [flank_half1, flank_half2]:
#                 right_flank_ref += str(half)
#         else:
#             right_flank_ref = fasta.get_seq(chrom, mut + 1, end_seq_loc_ref)
#         ref_seq1 = str(left_flank) + middle_ref + str(right_flank_ref)
#         assert len(ref_seq1) == 93
#         if strand == "-1":
#             ref_seq_dict[prot_id][0].append(complementary_sequence(ref_seq1))
#         else:
#             ref_seq_dict[prot_id][0].append(ref_seq1)
#         middle_var = var
#         # right side for variant
#         end_seq_loc_var = mut + (RIGHT_FRAME - len(var) + 3) + 1  # this is the pad
#         exon_end_diff_var = filtered_exon_pos[prot_id][0][1] - end_seq_loc_var
#         if exon_end_diff_var < 0:
#             right_flank_var = ""
#             exon_end_diff_var = (
#                 filtered_exon_pos[prot_id][0][1] - mut + len(var)
#             )  # + 1 for the mutation start location
#             end_seq_loc_var_right_exon = (
#                 filtered_exon_pos[prot_id][1][0] + RIGHT_FRAME - exon_end_diff_var
#             )
#             flank_half1 = fasta.get_seq(chrom, mut, filtered_exon_pos[prot_id][0][1])
#             flank_half2 = fasta.get_seq(
#                 chrom,
#                 filtered_exon_pos[prot_id][1][0],
#                 end_seq_loc_var_right_exon - len(var),
#             )
#             for half in [flank_half1, flank_half2]:
#                 right_flank_var += str(half)
#         else:
#             right_flank_var = fasta.get_seq(chrom, mut + 1, end_seq_loc_var)
#         var_seq1 = str(left_flank) + middle_var + str(right_flank_var)
#         assert len(var_seq1) == 93
#         if strand == "-1":
#             var_seq_dict[prot_id][0].append(complementary_sequence(var_seq1))
#         else:
#             var_seq_dict[prot_id][0].append(var_seq1)
#         # get right hang
#         overlap_ref = get_right_hang(
#             first_seq=ref_seq1,
#             chrom=chrom,
#             prot_id=prot_id,
#             first_seq_end_loc=end_seq_loc_ref,
#             filtered_exon_pos=filtered_exon_pos,
#             fasta=fasta,
#         )
#         overlap_var = get_right_hang(
#             first_seq=var_seq1,
#             chrom=chrom,
#             prot_id=prot_id,
#             first_seq_end_loc=end_seq_loc_var,
#             filtered_exon_pos=filtered_exon_pos,
#             fasta=fasta,
#         )
#         if strand == "-1":
#             overlap_ref = [complementary_sequence(s) for s in overlap_ref]
#             overlap_var = [complementary_sequence(s) for s in overlap_var]
#         ref_seq_dict[prot_id][0] += overlap_ref
#         var_seq_dict[prot_id][0] += overlap_var

In [38]:
filtered_exon_pos

defaultdict(list,
            {'ENSP00000369497.3': [(32379317, 32379515),
              (32379750, 32379913),
              (32380007, 32380145),
              (32394689, 32394933),
              (32396898, 32397044),
              (32398162, 32400266)]})

In [39]:
codon_ref == "-"

False

In [34]:
mut

32379450

In [35]:
mut2

32379451

In [33]:
frame_dict

NameError: name 'frame_dict' is not defined

In [24]:
exon_start_diff

133

In [22]:
len(var_seq1)

96

In [16]:
# def get_sequences_deletion(
#     mut_info,
#     exon_info,
#     fasta,
# ):
from collections import defaultdict

RIGHT_FRAME = 48
(
    # ref,
    var,
    # aa_ref,
    # aa_var,
    codon_ref,
    # codon_var,
    chrom,
    mut,
    mut2,
    # var_class,
    gene,
    # protein,
    # transcript,
    # gene_name,
    strand,
) = (
    # mut_info["reference"],
    mut_info["variant"],
    # mut_info["amino_acid_ref"],
    # mut_info["amino_acid_var"],
    mut_info["codon_ref"],
    # mut_info["codon_var"],
    mut_info["chromosome"],
    mut_info["mutation_location"],
    mut_info["mutation_location2"],
    # mut_info["variant_class"],
    mut_info["gene_id"],
    # mut_info["protein_id"],
    # mut_info["transcript_id"],
    # mut_info["gene_symbol"],
    mut_info["strand"],
)
fasta = refgen
filtered_exon_pos = filter_exon_pos(exon_info[gene], mut, mut_info)
ref_seq_dict, var_seq_dict = {}, {}
# This loop only works if codon is unknown. So the goal is to create 3 frames and the mutation is located on the 3' end with at least 45 bp on the left
if codon_ref == "-":
    for prot_id in filtered_exon_pos:
        ref_seq_dict[prot_id], var_seq_dict[prot_id] = {}, {}
        for i, mutx in enumerate([mut, mut2]):
            ref_seq_dict[prot_id][i], var_seq_dict[prot_id][i] = defaultdict(
                list
            ), defaultdict(list)
            exon_start_diff = mutx - filtered_exon_pos[prot_id][0][0]
            if exon_start_diff >= 47:
                left_frames = [45, 46, 47]
                frame_dict = {
                    45: 45,
                    46: 44,
                    47: 43,
                }
            else:
                left_frames = [
                    exon_start_diff,
                    exon_start_diff + 1,
                    exon_start_diff + 2,
                ]
                frame_dict = {
                    exon_start_diff: 90 - exon_start_diff,
                    exon_start_diff + 1: 90 - exon_start_diff - 1,
                    exon_start_diff + 2: 90 - exon_start_diff - 2,
                }
            for j, left_frame in enumerate(left_frames):
                # left side
                start_seq_loc = mutx - left_frame - 1
                left_flank = fasta.get_seq(
                    chrom,
                    start_seq_loc,
                    mutx,
                )
                while left_flank[-len(codon_ref) :] != codon_ref:
                    left_frame -= 1
                    start_seq_loc = mut - left_frame + 1
                    left_flank = fasta.get_seq(
                        chrom,
                        start_seq_loc,
                        mut,
                    )
                    if left_shifted:
                        if left_frame < 42:
                            break
                    else:
                        if left_frame < exon_start_diff - 3:
                            break
                # middle
                middle_ref = fasta.get_seq(
                    chrom,
                    mut + 1,
                    mut2,
                )
                middle_var = ""
                # right side for reference
                end_seq_loc_ref = (
                    mutx + frame_dict[left_frame]
                )  # because the location of the mutation counts as 1
                exon_end_diff_ref = filtered_exon_pos[prot_id][0][1] - end_seq_loc_ref
                if exon_end_diff_ref < 0:
                    right_flank_ref = ""
                    exon_end_diff_ref = (
                        filtered_exon_pos[prot_id][0][1] - mutx + 1
                    )  # + 1 for the mutation location itself
                    end_seq_loc_ref_right_exon = (
                        filtered_exon_pos[prot_id][1][0]
                        + frame_dict[left_frame]
                        - exon_end_diff_ref
                    )
                    flank_half1 = fasta.get_seq(
                        chrom, mutx, filtered_exon_pos[prot_id][0][1]
                    )
                    flank_half2 = fasta.get_seq(
                        chrom,
                        filtered_exon_pos[prot_id][1][0],
                        end_seq_loc_ref_right_exon,
                    )
                    for half in [flank_half1, flank_half2]:
                        right_flank_ref += str(half)
                else:
                    right_flank_ref = fasta.get_seq(chrom, mutx, end_seq_loc_ref)
                ref_seq1 = str(left_flank) + str(middle_ref) + str(right_flank_ref)
                assert len(ref_seq1) == 93
                if strand == "-1":
                    ref_seq_dict[prot_id][i][j].append(complementary_sequence(ref_seq1))
                else:
                    ref_seq_dict[prot_id][i][j].append(ref_seq1)
                # right side for variant
                end_seq_loc_var = mutx + (
                    frame_dict[left_frame] - len(var)
                )  # this is the pad
                exon_end_diff_var = filtered_exon_pos[prot_id][0][1] - end_seq_loc_var
                if exon_end_diff_var < 0:
                    right_flank_var = ""
                    exon_end_diff_var = (
                        filtered_exon_pos[prot_id][0][1] - mutx + len(var)
                    )  # + 1 for the mutation start location
                    end_seq_loc_var_right_exon = (
                        filtered_exon_pos[prot_id][1][0]
                        + frame_dict[left_frame]
                        - exon_end_diff_var
                    )
                    flank_half1 = fasta.get_seq(
                        chrom, mutx, filtered_exon_pos[prot_id][0][1]
                    )
                    flank_half2 = fasta.get_seq(
                        chrom,
                        filtered_exon_pos[prot_id][1][0],
                        end_seq_loc_var_right_exon - len(var),
                    )
                    for half in [flank_half1, flank_half2]:
                        right_flank_var += str(half)
                else:
                    right_flank_var = fasta.get_seq(chrom, mutx, end_seq_loc_var)
                var_seq1 = str(left_flank) + middle_var + str(right_flank_var)
                assert len(var_seq1) == 93
                if strand == "-1":
                    var_seq_dict[prot_id][i][j].append(complementary_sequence(var_seq1))
                else:
                    var_seq_dict[prot_id][i][j].append(var_seq1)
                # get right hang
                overlap_ref = get_right_hang(
                    first_seq=ref_seq1,
                    chrom=chrom,
                    prot_id=prot_id,
                    first_seq_end_loc=end_seq_loc_ref,
                    filtered_exon_pos=filtered_exon_pos,
                    fasta=fasta,
                )
                overlap_var = get_right_hang(
                    first_seq=var_seq1,
                    chrom=chrom,
                    prot_id=prot_id,
                    first_seq_end_loc=end_seq_loc_var,
                    filtered_exon_pos=filtered_exon_pos,
                    fasta=fasta,
                )
                if strand == "-1":
                    overlap_ref = [complementary_sequence(s) for s in overlap_ref]
                    overlap_var = [complementary_sequence(s) for s in overlap_var]
                ref_seq_dict[prot_id][i][j] += overlap_ref
                var_seq_dict[prot_id][i][j] += overlap_var
else:
    # this is now corect but should we add in a codon check?
    for prot_id in filtered_exon_pos:
        ref_seq_dict[prot_id], var_seq_dict[prot_id] = defaultdict(list), defaultdict(
            list
        )
        exon_start_diff = mut - filtered_exon_pos[prot_id][0][0]
        if exon_start_diff >= 47:
            left_frame = 45
            left_shifted = False
        else:
            left_frame = exon_start_diff
            left_shifted = True
        # left side
        start_seq_loc = mut2 - left_frame
        left_n, right_n = flanking_lower_positions(codon_ref)
        pad_right = left_n if strand == "-1" else right_n
        pad_left = left_n if strand == "-1" else right_n
        start_seq_loc -= pad_left
        left_flank_ref = fasta.get_seq(
            chrom,
            start_seq_loc,
            mut2 + pad_right,
        )
        check_left = (
            reverse_complement(str(left_flank[-len(codon_ref) :]))
            if strand == "-1"
            else str(left_flank[-len(codon_ref) :])
        )
        while check_left != codon_ref.upper():
            left_frame -= 1
            start_seq_loc = mut2 - left_frame + 1 - pad_left
            left_flank = fasta.get_seq(
                chrom,
                start_seq_loc,
                mut2 + pad_right,
            )
            if left_shifted:
                if left_frame < 42:
                    break
            else:
                if left_frame < exon_start_diff - 3:
                    break
        frame_shift_pad = 45 - left_frame
        # check the last 3 sequence to ensure that it's the same as codon_ref
        middle_var = ""
        # right side for reference
        end_seq_loc_ref = mut + RIGHT_FRAME + frame_shift_pad
        exon_end_diff_ref = filtered_exon_pos[prot_id][0][1] - end_seq_loc_ref
        if exon_end_diff_ref < 0:
            right_flank_ref = ""
            exon_end_diff_ref = (
                filtered_exon_pos[prot_id][0][1] - mut + 1
            )  # + 1 for the mutation location itself
            end_seq_loc_ref_right_exon = (
                filtered_exon_pos[prot_id][1][0]
                + RIGHT_FRAME
                + frame_shift_pad
                - exon_end_diff_ref
                + 1
            )
            flank_half1 = fasta.get_seq(chrom, mut, filtered_exon_pos[prot_id][0][1])
            flank_half2 = fasta.get_seq(
                chrom,
                filtered_exon_pos[prot_id][1][0],
                end_seq_loc_ref_right_exon,
            )
            for half in [flank_half1, flank_half2]:
                right_flank_ref += str(half)
        else:
            right_flank_ref = fasta.get_seq(chrom, mut + 1, end_seq_loc_ref)
        ref_seq1 = str(left_flank) + str(right_flank_ref)
        assert len(ref_seq1) == 93
        if strand == "-1":
            ref_seq_dict[prot_id][0].append(complementary_sequence(ref_seq1))
        else:
            ref_seq_dict[prot_id][0].append(ref_seq1)
        # right side for variant
        end_seq_loc_var = mut + (RIGHT_FRAME - len(var) + 3) + 1  # this is the pad
        exon_end_diff_var = filtered_exon_pos[prot_id][0][1] - end_seq_loc_var
        if exon_end_diff_var < 0:
            right_flank_var = ""
            exon_end_diff_var = (
                filtered_exon_pos[prot_id][0][1] - mut + len(var)
            )  # + 1 for the mutation start location
            end_seq_loc_var_right_exon = (
                filtered_exon_pos[prot_id][1][0] + RIGHT_FRAME - exon_end_diff_var
            )
            flank_half1 = fasta.get_seq(chrom, mut, filtered_exon_pos[prot_id][0][1])
            flank_half2 = fasta.get_seq(
                chrom,
                filtered_exon_pos[prot_id][1][0],
                end_seq_loc_var_right_exon - len(var),
            )
            for half in [flank_half1, flank_half2]:
                right_flank_var += str(half)
        else:
            right_flank_var = fasta.get_seq(chrom, mut + 1, end_seq_loc_var)
        var_seq1 = str(left_flank) + middle_var + str(right_flank_var)
        assert len(var_seq1) == 93
        if strand == "-1":
            var_seq_dict[prot_id][0].append(complementary_sequence(var_seq1))
        else:
            var_seq_dict[prot_id][0].append(var_seq1)
        # get right hang
        overlap_ref = get_right_hang(
            first_seq=ref_seq1,
            chrom=chrom,
            prot_id=prot_id,
            first_seq_end_loc=end_seq_loc_ref,
            filtered_exon_pos=filtered_exon_pos,
            fasta=fasta,
        )
        overlap_var = get_right_hang(
            first_seq=var_seq1,
            chrom=chrom,
            prot_id=prot_id,
            first_seq_end_loc=end_seq_loc_var,
            filtered_exon_pos=filtered_exon_pos,
            fasta=fasta,
        )
        if strand == "-1":
            overlap_ref = [complementary_sequence(s) for s in overlap_ref]
            overlap_var = [complementary_sequence(s) for s in overlap_var]
        ref_seq_dict[prot_id][0] += overlap_ref
        var_seq_dict[prot_id][0] += overlap_var
    # return ref_seq_dict, var_seq_dict

NameError: name 'left_flank' is not defined

In [25]:
len(left_flank)

48

In [26]:
left_flank

>chr1:19155621-19155668
GAGAAAATAGAAGGTTGGACTCAAGGCATCAACACACAGGTCATAAAT

In [27]:
flank_half1

>chr1:19155665-19155668
AAAT

In [23]:
len(flank_half2)

46

In [21]:
flank_half2

>chr1:19210073-19210118
CTCTCGATGACTGAGGCCACCAGCTGCGGCAACTCCTTCATCTCGA

In [133]:
reverse_complement(str(left_flank[-len(codon_ref) :])) == codon_ref.upper()

True

In [130]:
codon_ref

'aTTTat'

In [128]:
complementary_sequence(_codon_ref)

'aTTTat'

In [118]:
codon_ref

'aTTTat'

In [120]:
left_flank

>chr1:19155624-19155668
AAAATAGAAGGTTGGACTCAAGGCATCAACACACAGGTCATAAAT

In [124]:
def reverse_complement(dna_sequence):
    complement_dict = {"A": "T", "T": "A", "C": "G", "G": "C"}
    reverse_comp_seq = "".join(
        complement_dict.get(base, base) for base in reversed(dna_sequence)
    )
    return reverse_comp_seq

In [114]:
def flanking_lower_positions(string):
    first_upper_index = None
    last_upper_index = None

    for i, char in enumerate(string):
        if char.isupper():
            if first_upper_index is None:
                first_upper_index = i
            last_upper_index = i

    left_count = (
        sum(1 for i in range(first_upper_index) if string[i].islower())
        if first_upper_index is not None
        else 0
    )
    right_count = (
        sum(1 for i in range(last_upper_index + 1, len(string)) if string[i].islower())
        if last_upper_index is not None
        else 0
    )

    return left_count, right_count

In [115]:
flanking_lower_positions(codon_ref)

(1, 2)